In [5]:
#import all our needed libraries and modules here
import keras
import tensorflow as tf
from keras.models import Sequential
from keras import backend as K
from keras.layers.core import Dense, Activation,Dropout
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import Model
from keras.applications import imagenet_utils
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications.mobilenet import MobileNet
from keras.applications.mobilenet import preprocess_input
import numpy as np
import pandas as pd
from IPython.display import Image
from keras.optimizers import Adam
from matplotlib import pyplot as plt
# from keras.activations import rmsprop 


In [6]:
np.random.seed(55)

In [7]:
train_path='./CervicalCancer/Train'
test_path='./CervicalCancer/Test'
valid_path='./CervicalCancer/Valid'

In [8]:
train_samples = 735
test_samples=91
validation_samples = 91
epochs = 40
batch_size = 16
img_classes=['Normal','Abnormal']
img_target_size=(224,224)

In [9]:
#generating batches of images in tensor form which is what will be feed into our network

#generating train batches
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
train_batches=train_datagen.flow_from_directory(train_path,target_size=img_target_size,batch_size=10)

FileNotFoundError: ignored

In [ ]:
#generating test batches
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_batches=test_datagen.flow_from_directory(test_path,target_size=img_target_size,batch_size=6)

In [ ]:
#generating valid batches
valid_batches=test_datagen.flow_from_directory(valid_path,target_size=img_target_size,batch_size=6)

In [ ]:
from keras.applications import mobilenet
#importing the mobilenetV2 model and discarding the last 1000 neurons
mobile =MobileNet()


In [11]:
mobile.summary()

Model: "mobilenet_1.00_224"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 112, 112, 32)     128       
 ation)                                         

In [ ]:
x=mobile.layers[-6].output
prediction=Dense(2,activation='softmax')(x)
model=Model(inputs=mobile.input, outputs=prediction)

In [ ]:
model.summary()

In [ ]:
for layer in mobile.layers[:-10]:
    layer.trainable=False

In [ ]:
layers = [(layer, layer.name, layer.trainable) for layer in mobile.layers]
layer=pd.DataFrame(layers, columns=['Layer Type', 'Layer Name', 'Layer Trainable'])

In [ ]:
lay = [(layer, layer.name, layer.trainable) for layer in model.layers]
laye=pd.DataFrame(layers, columns=['Layer Type', 'Layer Name', 'Layer Trainable'])

In [ ]:
laye

In [ ]:
layer[:-8]

In [ ]:
#now compiliing our new model
model.compile(Adam(lr=0.0001),loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
history=model.fit_generator(train_batches,steps_per_epoch=74,
                    validation_data=valid_batches,validation_steps=16,epochs=epochs,verbose=2,
                           shuffle=True)

In [ ]:
_, acc = model.evaluate_generator(test_batches, steps=len(test_batches), verbose=2)
print('Test Accuracy: %.3f' % (acc * 100))

In [2]:
# Plotting training & validation accuracy values
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train_acc', 'val_acc'], loc='upper left')
plt.show()

NameError: ignored

In [ ]:
# Plotting training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train_loss', 'val_loss'], loc='upper left')
plt.show()